<a href="https://colab.research.google.com/github/epg900/v2ray/blob/main/V2Ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title V2Ray Server
import json
import base64
import uuid
import os

!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!wget https://github.com/v2fly/v2ray-core/releases/download/v4.45.2/v2ray-linux-64.zip
!dpkg -i cloudflared-linux-amd64.deb
!unzip v2ray-linux-64.zip
!pip install pyqrcode

os.remove("config.json")
idx=str(uuid.uuid4())
!chmod +x v2ray
CONFIG_JSON1="{\"log\":{\"access\":\"\",\"error\":\"\",\"loglevel\":\"warning\"},\"inbound\":{\"protocol\":\"vmess\",\"port\":"
CONFIG_JSON2=",\"settings\":{\"clients\":[{\"id\":\""
CONFIG_JSON3="\",\"alterId\":64}]},\"streamSettings\":{\"network\":\"ws\"}},\"inboundDetour\":[],\"outbound\":{\"protocol\":\"freedom\",\"settings\":{}}}"
with open("config.json", "w") as f:
  f.write(CONFIG_JSON1+str(9910)+CONFIG_JSON2+idx+CONFIG_JSON3)

from threading import Timer
from subprocess import Popen , PIPE
import re, time , base64
import pyqrcode
from IPython.display import HTML, clear_output

Popen("./v2ray run".split(), cwd='./', env={'V2RAY_VMESS_AEAD_FORCED':'false'})
Popen("cloudflared tunnel --url 127.0.0.1:9910 --logfile cloudflared.log".split(), stdout=PIPE, stdin=PIPE, stderr=PIPE, universal_newlines=True)
time.sleep(5)
ff=open("cloudflared.log", "r")
txt=ff.read()
ff.close()
addr=re.findall("https://(.*?.trycloudflare.com)",txt)

d=json.loads('{"add":"{0}","aid":"64","host":"","id":"{1}","net":"ws","path":"","port":"80","ps":"1","tls":"","type":"none","v":"2"}')
d["add"] = addr[0]
d["id"] = idx

clear_output()

print(addr[0])
print(idx)
print(json.dumps(d))

config="vmess://"+base64.b64encode(json.dumps(d).encode()).decode("utf-8")
url = pyqrcode.create(config)
url.svg('qrcode.svg', scale=8)
imgfile=base64.b64encode(open("qrcode.svg","rb").read()).decode('ascii')
display(HTML("<center><img width='270px' height='270px'  src='data:image/svg+xml;base64,{}' /></center>".format(imgfile)))
time.sleep(7200)
